## XGBデフォルト

In [1]:
import numpy as np
import pandas as pd

import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, make_scorer, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk3')

In [3]:
train_pkl.shape

(850, 10)

In [4]:
# 判断にあまり使用しなかった下位３つ削除
# ALP, TP, Alb, Age を削除

In [5]:
train_pkl = train_pkl.drop(['ALP', 'TP', 'Alb', 'Age'], axis=1)

In [6]:
train_pkl.shape

(850, 6)

## 訓練データとテストデータに分割

In [7]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [8]:
# ターゲットと特徴量の分割
train_X = train.iloc[:, :-1]
train_y = train.disease.values

In [9]:
train_X.head(5)

,T_Bil,D_Bil,ALT_GPT,AST_GOT,AG_ratio
332,2.066406,0.631348,14.563273,47.154297,0.759766
383,0.817871,0.197021,15.622564,21.059477,1.291016
281,0.791992,0.082642,12.924613,25.772480,1.319336
2,0.657227,0.081299,12.625011,30.613180,0.774902
231,1.714844,0.511230,27.669710,60.709866,1.218750


## 訓練データで訓練

In [10]:
XGB = XGBClassifier()

In [11]:
XGB = XGB.fit(train_X, train_y)

In [12]:
XGB.feature_importances_

array([0.32739106, 0.1332557 , 0.19395043, 0.21917538, 0.12622747],
      dtype=float32)

In [13]:
sorted(
    zip(map(lambda x: round(x, 3), XGB.feature_importances_), train.iloc[:, :-1].columns),
    reverse=True)

[(0.327, 'T_Bil'),
 (0.219, 'AST_GOT'),
 (0.194, 'ALT_GPT'),
 (0.133, 'D_Bil'),
 (0.126, 'AG_ratio')]

## テストデータで実行

In [14]:
# ターゲットと特徴量の分割
test_x = test.iloc[:, :-1]
test_y = test.disease.values

In [15]:
test_x.shape, test_y.shape

((170, 5), (170,))

In [16]:
pred_y = XGB.predict(test_x)

In [17]:
confusion_matrix(test_y, pred_y)

array([[85,  8],
       [14, 63]], dtype=int64)

In [18]:
accuracy_score(test_y, pred_y)

0.8705882352941177

## 検証データで実行

In [ ]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk3')

In [ ]:
valid.shape

In [ ]:
type(valid)

In [ ]:
# ID の保存
# valid_pass = valid.id.values

In [ ]:
# valid_X = valid.iloc[:, 1:]
valid_X = valid.copy()

In [ ]:
valid_X.shape, train_X.shape

In [ ]:
pred_valid_y = XGB.predict(valid_X)

In [ ]:
pred_valid_y.shape

In [ ]:
# type(valid_pass), type(pred_valid_y)

In [ ]:
result_df = pd.DataFrame(pred_valid_y)

In [ ]:
result_df.to_csv("./XGB_4.csv", header=False)